### [ Model save & load ]
- 2가지 형태 저장
    * 전체 저장 
    * 모델의 파라미터만 저장 
- 2가지 형태 로딩
    * 전체 저장 모델 파일 ==> 로딩으로 사용 가능
    * 모델 파라미터 저장 파일 ==> 모델 인스턴스 생성 후 층별 파라미터 적용

[1] 모듈 로딩 및 데이터 준비<hr>

In [2]:
## [1-1] 모듈 로딩

import os, sys
sys.path.append(os.path.abspath(".."))

import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 
from   torch.optim.lr_scheduler import ReduceLROnPlateau

from   torch.utils.data import Dataset, TensorDataset, DataLoader
from   torch.utils.data import random_split
from   sklearn.model_selection import train_test_split

from Utils import util_func as uf
import pandas as pd

## 전역 재현성 설정
torch.manual_seed(10)

In [2]:
## [1-2] 데이터 준비
DATA_FILE = '../Data/study_score_multi.csv'

dataDF    = pd.read_csv(DATA_FILE)
featureDF = dataDF[dataDF.columns[:-1]]
targetDF  = dataDF[dataDF.columns[-1:]]


In [3]:
## [1-3] 데이터셋 클래스 정의
class TestDataset(Dataset):

    def __init__(self, featureDF, targetDF):
        super().__init__()
        self.xTS = torch.tensor(featureDF.values, dtype=torch.float32)
        self.yTS = torch.tensor(targetDF.values, dtype=torch.float32) 
        self.length = featureDF.shape[0]

    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        return self.xTS[idx], self.yTS[idx]
    

In [4]:
## 학습용/검증용/테스트용 데이터셋 분리
x_train, x_test, y_train, y_test   = train_test_split(featureDF, targetDF,
                                                      test_size=0.2,
                                                      random_state=10)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train,
                                                      test_size=0.2,
                                                      random_state=10)
trainDS  = TestDataset(x_train, y_train)
validDS  = TestDataset(x_valid, y_valid)
testDS   = TestDataset(x_test, y_test)

print(len(trainDS), len(validDS), len(testDS))

3200 800 1000


[2] 모델 클래스 정의<hr>

In [5]:
class Test(nn.Module):
    def __init__(self, in_out, out_out):
        super().__init__()
        self.fc1 = nn.Linear(in_out, 5)
        self.fc2 = nn.Linear(5, 7)
        self.out = nn.Linear(7, out_out)

    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = F.relu(self.fc2(out))
        return self.out(out)

[3] 학습 진행<hr>

In [6]:
## 설정값들 
EPOCHS = 1000
BS = 500
LR = 0.1

## 저장 모델 파일명  => 
ALL_MODEL     = '../Models/RegScore_model.pt'    ## 모델 전체 확장자    pt
WEIGHTS_MODEL = '../Models/RegScore_weights'     ## 파라미터 저장 확장자 pth

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


In [7]:
## 인스턴스들
from torchmetrics.regression import R2Score, MeanAbsoluteError, MeanSquaredError 
from torchmetrics.classification import F1Score, Recall
model       = Test(3, 1).to(DEVICE)
loss_fn     = nn.MSELoss()
matrics_fn  = R2Score() 
optimizer   = optim.Adam(model.parameters(), lr= LR)

trainDL     = DataLoader(trainDS, batch_size=BS, shuffle=True)
validDL     = DataLoader(validDS, batch_size=BS) 
testDL      = DataLoader(testDS,  batch_size=BS)

## 스케쥴러 : 검증데이터의 성능지표에 대한 기준 loss, acc
scheduler   = ReduceLROnPlateau(optimizer, mode='min', patience=3)



In [8]:
## 학습진행 =========================
BEST_LOSS       = 100.
EARLY_STOP_CNT  = 10

for epoch in range(EPOCHS):
    #- 학습진행
    train_loss, train_r2, train_mae = uf.train_one_epoch_reg(model, trainDL, loss_fn, optimizer, DEVICE)

    #- 검증진행
    valid_loss, valid_r2, valid_mae = uf.evaluate_reg(model, validDL, loss_fn, DEVICE)

    #- 모델과 가중치 파일 저장
    if BEST_LOSS > valid_loss:
        #- 모델 전체 저장
        torch.save(model, ALL_MODEL)
        #- 파라미터만 저장
        torch.save(model.state_dict(), f"./{WEIGHTS_MODEL}_{epoch:03}_{valid_loss:.5f}.pth")
        #- 기준 loss 점수 업데이트
        BEST_LOSS = valid_loss

    #- 에포크마다 스케쥴러에게 검증성능 업데이트 
    scheduler.step(valid_loss)

    #- 학습상태 출력
    print(f"{epoch}_[LOSS] train:valid={train_loss:.5f}:{valid_loss:.5f} ", end=' ')
    print(f"[ACC] train:valid={train_r2:.5f}:{valid_r2:.5f} ")
    print(f"[LR] {scheduler.get_last_lr()[0]:.5f} bad_epochs : {scheduler.num_bad_epochs}")

    #- 조기종료 체크
    if(scheduler.patience == scheduler.num_bad_epochs):
        EARLY_STOP_CNT -= 1
    
    if not EARLY_STOP_CNT: 
        print("성능 개선이 없어 조기종료합니다.")
        break


0_[LOSS] train:valid=1608.87847:438.62326  [ACC] train:valid=-2.63032:-0.01058 
[LR] 0.10000 bad_epochs : 0
1_[LOSS] train:valid=552.87957:305.34132  [ACC] train:valid=-0.24753:0.29650 
[LR] 0.10000 bad_epochs : 0
2_[LOSS] train:valid=277.87538:205.40114  [ACC] train:valid=0.37299:0.52676 
[LR] 0.10000 bad_epochs : 0
3_[LOSS] train:valid=146.43783:115.61910  [ACC] train:valid=0.66957:0.73362 
[LR] 0.10000 bad_epochs : 0
4_[LOSS] train:valid=95.38910:77.29597  [ACC] train:valid=0.78476:0.82191 
[LR] 0.10000 bad_epochs : 0
5_[LOSS] train:valid=72.80601:65.34951  [ACC] train:valid=0.83572:0.84944 
[LR] 0.10000 bad_epochs : 0
6_[LOSS] train:valid=55.19903:48.34573  [ACC] train:valid=0.87545:0.88861 
[LR] 0.10000 bad_epochs : 0
7_[LOSS] train:valid=46.59577:47.57514  [ACC] train:valid=0.89486:0.89039 
[LR] 0.10000 bad_epochs : 0
8_[LOSS] train:valid=41.24207:40.17885  [ACC] train:valid=0.90694:0.90743 
[LR] 0.10000 bad_epochs : 0
9_[LOSS] train:valid=38.85788:39.27092  [ACC] train:valid=0.9

[4] 모델 파일 사용 <hr>

In [ ]:
## [4-1] 가중치 저장 파일 로딩
params = torch.load("./weights_004_0.18401.pth", weights_only=True)

tModel = Test()
tModel.load_state_dict(params)

In [ ]:
## [4-2] 전체 모델 저장 파일 로딩
allModel = torch.load(ALL_MODEL, weights_only=False)
allModel